# Personal Budget Data Preparation

## How to Generate Mock Data

Visit https://www.mockaroo.com to generate the data there, delete the already existing fields and add the following field for each table with the following settings:

**Note**: Make sure all fields' blank value set to 0% except the notes field you can play with the blank value from 60% to 80%, more or less.

### income_log

After generating the data as **CSV**, create another table for user 2, play with the content a bit to make it different.

|Field     | Type | Options               |
|:--------:|:----:|:---------------------|
|user_id|Number|min:**1**,max:**1**, switch both numbers to **2** when creating next user's table|
|date|Datetime|**1/1/2024** to **now**, format: **yyyy-mm-dd**|
|source|Custom List|Content: **Event Organization, Delivery Job, Other**, switch **random** to **weighted**, click the button after weighted and give each item the following weight: **Event Organization: 7**, **Delivery Job: 10**, **Other: 2**|
|amount|Number|min:**50**, max:**120**|
|notes|Sentences|At least **1** but no more than **1**, blank: **80%**

After downloading the CSV file, give it the name `raw_income_log_01.csv` and `raw_income_log_02.csv` where 1 is for user 1 and 2 is for user 2.

### savings_log

After generating the data as **CSV**, create another table for user 2, play with the content a bit to make it different.


|Field     | Type | Options               |
|:--------:|:----:|:---------------------|
|user_id|Number|min:**1**,max:**1**, switch both numbers to **2** when creating next user's table|
|date|Datetime|**1/1/2024** to **now**, format: **yyyy-mm-dd**|
|change|Number|min:**-100**, max:**120**|
|notes|Sentences|At least **1** but no more than **1**, blank: **80%**


After downloading the CSV file, give it the name `raw_savings_log_01.csv` and `raw_savings_log_02.csv` where 1 is for user 1 and 2 is for user 2.

**Note**: I had to generate the data multiple times to get it right. After generating the data, sum the change column and make sure it's not negative, nor very high.

### investments_log

For this table, set the **Number of Rows** to around **600**. After generating the data as **CSV**, create another table for user 2, play with the content a bit to make it different.


|Field     | Type | Options               |
|:--------:|:----:|:---------------------|
|user_id|Number|min:**1**,max:**1**, switch both numbers to **2** when creating next user's table|
|date|Datetime|**1/8/2024** to **now**, format: **yyyy-mm-dd**|
|change|Number|min:**-70**, max:**70**|
|notes|Sentences|At least **1** but no more than **1**, blank: **50%**


After downloading the CSV file, give it the name `raw_investments_log_01.csv` and `raw_investments_log_02.csv` where 1 is for user 1 and 2 is for user 2.

**Note**: I had to generate the data multiple times to get it right. After generating the data, sum the change column and make sure it's not negative, nor very high.

### debts_log

For this table, we'll be creating **two tables per user**. One with the `type` column filled with "Credited" and the other filled with "Recieved". After generating the data as **CSV**, we'll create another two tables for user 2, play with the content a bit to make it different. Also, set the **number of rows per table** to around **50**

|Field     | Type | Options               |
|:--------:|:----:|:---------------------|
|user_id|Number|min:**1**,max:**1**, switch both numbers to **2** when creating next user's table|
|date|Datetime|**1/1/2024** to **now**, format: **yyyy-mm-dd**|
|party|Custom List|Content: **Ahmad, Issa, Sarah, Mila**, for this keep it random. Also, after generating the table, change up the name for other tables|
|amount|Number|min:**-60**, max:**80**|
|notes|Sentences|At least **1** but no more than **1**, blank: **75%**
|type|Custom List| Content: **Credited**, after generating this table, change the column to **Recieved**

After downloading the CSV file, give it the name `raw_debts_log_01.csv` and `raw_debts_log_02.csv` for user 1. And `raw_debts_log_03.csv` and `raw_debts_log_04.csv` for user two. Make sure tables 1 and 3 have the `type` column set to `Credited`, and tables 2 and 4 have the `type` column set to `Recieved`.

**Note**: I had to generate the data multiple times to get it right. After generating the data, sum the amount per party and make sure none are negative, nor very high.

### subscriptions

For this table, I'll be creating it manually with python since it will contain around 3-6 rows only, and the data has to be specific and realistic.

## Data Preparation Plan

### income_log table

1. Import mock income_log data (two)
2. Combine dataframes to one dataframe
3. Sort resulting dataframe by date
4. View total income per month per user
5. If monthly totals are not realistic, change mock data
6. Remove ";" from values in `notes` column
7. Export data as sql file with prefix `final_`

### savings_log table

1. Import mock savings_log data (two)
2. Sort each dataframe by date
3. Calculate sum of `change` column per dataframe
4. If sum is negative, change mock data
5. Add new cumulative column called `balance` for each dataframe using `cumsum`
6. Drop rows where `balance` is negative
7. Combine dataframes to one dataframe
8. Try: Move `balance` column to be after `change` column
9. Sort resulting dataframe by date
10. Remove ";" from values in `notes` column
11. Export dataframe to sql file with prefix `final_`

### users table

Write sql file containing two users each with `name`, `username`, and `password`

## Implementation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### income_log table

#### 1. Import mock income_log data (two)

In [2]:
income_log_1 = pd.read_csv("raw_income_log_01.csv")
income_log_2 = pd.read_csv("raw_income_log_02.csv")

In [3]:
income_log_1

,user_id,date,source,amount,notes
0,1,2025-04-25,Delivery Job,24,NaN
1,1,2025-01-06,Event Organizing,91,NaN
2,1,2024-04-29,Event Organizing,86,NaN
3,1,2025-06-21,Delivery Job,25,NaN
4,1,2024-02-03,Delivery Job,82,NaN
...,...,...,...,...,...
995,1,2025-06-15,Delivery Job,51,NaN
996,1,2025-05-10,Event Organizing,64,NaN
997,1,2024-06-09,Delivery Job,96,NaN
998,1,2024-12-03,Delivery Job,16,NaN


In [4]:
income_log_2

,user_id,date,source,amount,notes
0,2,2024-05-14,University,105,NaN
1,2,2025-03-22,Other,55,NaN
2,2,2024-01-16,University,109,NaN
3,2,2024-01-03,University,112,NaN
4,2,2025-03-27,University,64,NaN
...,...,...,...,...,...
995,2,2025-06-11,University,66,NaN
996,2,2024-09-05,Shop,105,NaN
997,2,2024-10-19,University,71,NaN
998,2,2024-10-29,Shop,120,Integer tincidunt ante vel ipsum.


#### 2. Combine dataframes to one dataframe

In [5]:
income_log = pd.concat([income_log_1, income_log_2], ignore_index=True)

In [6]:
income_log

,user_id,date,source,amount,notes
0,1,2025-04-25,Delivery Job,24,NaN
1,1,2025-01-06,Event Organizing,91,NaN
2,1,2024-04-29,Event Organizing,86,NaN
3,1,2025-06-21,Delivery Job,25,NaN
4,1,2024-02-03,Delivery Job,82,NaN
...,...,...,...,...,...
1995,2,2025-06-11,University,66,NaN
1996,2,2024-09-05,Shop,105,NaN
1997,2,2024-10-19,University,71,NaN
1998,2,2024-10-29,Shop,120,Integer tincidunt ante vel ipsum.


#### 3. Sort resulting dataframe by date

In [7]:
# Convert date column to datetime
income_log["date"] = pd.to_datetime(income_log["date"])

In [8]:
# Sort
income_log = income_log.sort_values("date", ascending=True)

In [9]:
# Reset dataframe index
income_log = income_log.reset_index(drop=True)

In [10]:
income_log

,user_id,date,source,amount,notes
0,2,2024-01-01,Other,90,NaN
1,2,2024-01-01,Shop,70,NaN
2,1,2024-01-01,Event Organizing,96,NaN
3,2,2024-01-01,University,89,NaN
4,1,2024-01-02,Delivery Job,83,NaN
...,...,...,...,...,...
1995,2,2025-08-23,University,59,Nam tristique tortor eu pede.
1996,2,2025-08-24,University,111,NaN
1997,2,2025-08-25,University,74,NaN
1998,2,2025-08-25,Other,76,NaN


#### 4. View total income per month per user

In [11]:
# Split dataframes by user
income_log_1 = income_log[income_log["user_id"] == 1]
income_log_2 = income_log[income_log["user_id"] == 2]

In [12]:
# Group sum of amount by month
monthly_totals_1 = income_log_1.groupby(income_log_1["date"].dt.to_period("M"))["amount"].sum()
monthly_totals_2 = income_log_2.groupby(income_log_2["date"].dt.to_period("M"))["amount"].sum()

In [13]:
monthly_totals_1

date
2024-01    2197
2024-02    2370
2024-03    2573
2024-04    2872
2024-05    3134
2024-06    3126
2024-07    3023
2024-08    3471
2024-09    2711
2024-10    1942
2024-11    3423
2024-12    2722
2025-01    3471
2025-02    2801
2025-03    2192
2025-04    2967
2025-05    2864
2025-06    2512
2025-07    3510
2025-08    2327
Freq: M, Name: amount, dtype: int64

In [14]:
monthly_totals_2

date
2024-01    4292
2024-02    4996
2024-03    5988
2024-04    4258
2024-05    4094
2024-06    4275
2024-07    4282
2024-08    4484
2024-09    3897
2024-10    4474
2024-11    3285
2024-12    4027
2025-01    4312
2025-02    4774
2025-03    4255
2025-04    4044
2025-05    3798
2025-06    3577
2025-07    4362
2025-08    2801
Freq: M, Name: amount, dtype: int64

#### 5. If monthly totals are not realistic, change mock data

Data looks realistic enough.

#### 6. Remove ";" from values in `notes` column

SchemaLoader in the main app splits sql statements by ';' so keeping it in notes values causes an error.

In [15]:
income_log.loc[income_log["notes"].notna(), "notes"] = (
    income_log.loc[income_log["notes"].notna(), "notes"]
    .str.replace(";", "", regex=False)
)

#### 7. Export data as sql file with prefix `final_`

In [16]:
# Convert income_log date column back to string
income_log["date"] = income_log["date"].dt.strftime("%Y-%m-%d")

In [17]:
# Write to sql file
with open("final_income_log.sql", "w", encoding="UTF-8") as file:
    for _, row in income_log.iterrows():
        notes = 'NULL' if str(row['notes']) == 'nan' else f"'{row['notes']}'"
        sql = f"INSERT INTO income_log (user_id, date, source, amount, notes) VALUES ({row['user_id']}, '{row['date']}', '{row['source']}', {row['amount']}, {notes});\n"
        file.write(sql)

### savings_log table

#### 1. Import mock savings_log data (two)

In [18]:
savings_log_1 = pd.read_csv("raw_savings_log_01.csv")
savings_log_2 = pd.read_csv("raw_savings_log_02.csv")

In [19]:
savings_log_1

,user_id,date,change,notes
0,1,2025-05-25,-57,Aliquam sit amet diam in magna bibendum imperd...
1,1,2025-01-30,47,Phasellus sit amet erat.
2,1,2025-07-09,-38,NaN
3,1,2024-12-23,28,NaN
4,1,2025-06-17,91,NaN
...,...,...,...,...
995,1,2024-06-03,-31,Curabitur convallis.
996,1,2025-06-11,78,Morbi non quam nec dui luctus rutrum.
997,1,2024-10-22,54,NaN
998,1,2025-07-14,83,NaN


In [20]:
savings_log_2

,user_id,date,change,notes
0,2,2025-02-24,-100,NaN
1,2,2025-02-14,-82,Proin interdum mauris non ligula pellentesque ...
2,2,2024-07-25,-17,"Lorem ipsum dolor sit amet, consectetuer adipi..."
3,2,2024-04-10,72,Curabitur in libero ut massa volutpat convallis.
4,2,2024-11-01,-43,Vestibulum ac est lacinia nisi venenatis trist...
...,...,...,...,...
995,2,2024-01-01,-51,NaN
996,2,2025-02-26,31,Sed accumsan felis.
997,2,2024-06-14,102,NaN
998,2,2024-04-10,-34,NaN


#### 2. Sort each dataframe by date

In [21]:
# Convert date column to datetime for both
savings_log_1["date"] = pd.to_datetime(savings_log_1["date"])
savings_log_2["date"] = pd.to_datetime(savings_log_2["date"])

In [22]:
# Sort both by date
savings_log_1 = savings_log_1.sort_values("date")
savings_log_2 = savings_log_2.sort_values("date")

In [23]:
# Reset both indexes
savings_log_1 = savings_log_1.reset_index(drop=True)
savings_log_2 = savings_log_2.reset_index(drop=True)

In [24]:
savings_log_1

,user_id,date,change,notes
0,1,2024-01-01,15,Pellentesque viverra pede ac diam.
1,1,2024-01-01,-20,NaN
2,1,2024-01-01,39,"Lorem ipsum dolor sit amet, consectetuer adipi..."
3,1,2024-01-03,-52,Nullam varius.
4,1,2024-01-03,20,NaN
...,...,...,...,...
995,1,2025-08-21,-22,NaN
996,1,2025-08-22,-72,NaN
997,1,2025-08-23,91,Sed accumsan felis.
998,1,2025-08-24,8,"Maecenas leo odio, condimentum id, luctus nec,..."


In [25]:
savings_log_2

,user_id,date,change,notes
0,2,2024-01-01,25,Morbi porttitor lorem id ligula.
1,2,2024-01-01,-51,NaN
2,2,2024-01-01,29,In sagittis dui vel nisl.
3,2,2024-01-03,-54,NaN
4,2,2024-01-03,106,"Nulla neque libero, convallis eget, eleifend l..."
...,...,...,...,...
995,2,2025-08-23,-59,"Maecenas leo odio, condimentum id, luctus nec,..."
996,2,2025-08-23,21,NaN
997,2,2025-08-23,55,NaN
998,2,2025-08-23,-15,NaN


#### 3. Calculate sum of `change` column per dataframe

In [26]:
savings_log_1["change"].sum()

np.int64(1616)

In [27]:
savings_log_2["change"].sum()

np.int64(1990)

#### 4. If sum is negative, change mock data

I've changed the mock data a few times until I got the numbers right.

#### 5. Add new cumulative column called `balance` for each dataframe using `cumsum`

In [28]:
savings_log_1["balance"] = savings_log_1["change"].cumsum()
savings_log_2["balance"] = savings_log_2["change"].cumsum()

In [29]:
savings_log_1

,user_id,date,change,notes,balance
0,1,2024-01-01,15,Pellentesque viverra pede ac diam.,15
1,1,2024-01-01,-20,NaN,-5
2,1,2024-01-01,39,"Lorem ipsum dolor sit amet, consectetuer adipi...",34
3,1,2024-01-03,-52,Nullam varius.,-18
4,1,2024-01-03,20,NaN,2
...,...,...,...,...,...
995,1,2025-08-21,-22,NaN,1507
996,1,2025-08-22,-72,NaN,1435
997,1,2025-08-23,91,Sed accumsan felis.,1526
998,1,2025-08-24,8,"Maecenas leo odio, condimentum id, luctus nec,...",1534


In [30]:
savings_log_2

,user_id,date,change,notes,balance
0,2,2024-01-01,25,Morbi porttitor lorem id ligula.,25
1,2,2024-01-01,-51,NaN,-26
2,2,2024-01-01,29,In sagittis dui vel nisl.,3
3,2,2024-01-03,-54,NaN,-51
4,2,2024-01-03,106,"Nulla neque libero, convallis eget, eleifend l...",55
...,...,...,...,...,...
995,2,2025-08-23,-59,"Maecenas leo odio, condimentum id, luctus nec,...",1966
996,2,2025-08-23,21,NaN,1987
997,2,2025-08-23,55,NaN,2042
998,2,2025-08-23,-15,NaN,2027


#### 6. Drop rows where `balance` is negative

Checking how many rows have balance < 0

In [31]:
(savings_log_1["balance"] < 0).sum()

np.int64(238)

In [32]:
(savings_log_2["balance"] < 0).sum()

np.int64(4)

Dropping the rows

In [33]:
savings_log_1 = savings_log_1[savings_log_1["balance"] >= 0]
savings_log_2 = savings_log_2[savings_log_2["balance"] >= 0]

In [34]:
savings_log_1

,user_id,date,change,notes,balance
0,1,2024-01-01,15,Pellentesque viverra pede ac diam.,15
2,1,2024-01-01,39,"Lorem ipsum dolor sit amet, consectetuer adipi...",34
4,1,2024-01-03,20,NaN,2
15,1,2024-01-10,95,Aenean sit amet justo.,3
136,1,2024-03-08,86,Pellentesque eget nunc.,15
...,...,...,...,...,...
995,1,2025-08-21,-22,NaN,1507
996,1,2025-08-22,-72,NaN,1435
997,1,2025-08-23,91,Sed accumsan felis.,1526
998,1,2025-08-24,8,"Maecenas leo odio, condimentum id, luctus nec,...",1534


In [35]:
savings_log_2

,user_id,date,change,notes,balance
0,2,2024-01-01,25,Morbi porttitor lorem id ligula.,25
2,2,2024-01-01,29,In sagittis dui vel nisl.,3
4,2,2024-01-03,106,"Nulla neque libero, convallis eget, eleifend l...",55
5,2,2024-01-04,52,Nullam molestie nibh in lectus.,107
6,2,2024-01-05,-54,Cras pellentesque volutpat dui.,53
...,...,...,...,...,...
995,2,2025-08-23,-59,"Maecenas leo odio, condimentum id, luctus nec,...",1966
996,2,2025-08-23,21,NaN,1987
997,2,2025-08-23,55,NaN,2042
998,2,2025-08-23,-15,NaN,2027


#### 7. Combine dataframes to one dataframe

In [36]:
savings_log = pd.concat([savings_log_1, savings_log_2], ignore_index=True)

In [37]:
savings_log

,user_id,date,change,notes,balance
0,1,2024-01-01,15,Pellentesque viverra pede ac diam.,15
1,1,2024-01-01,39,"Lorem ipsum dolor sit amet, consectetuer adipi...",34
2,1,2024-01-03,20,NaN,2
3,1,2024-01-10,95,Aenean sit amet justo.,3
4,1,2024-03-08,86,Pellentesque eget nunc.,15
...,...,...,...,...,...
1753,2,2025-08-23,-59,"Maecenas leo odio, condimentum id, luctus nec,...",1966
1754,2,2025-08-23,21,NaN,1987
1755,2,2025-08-23,55,NaN,2042
1756,2,2025-08-23,-15,NaN,2027


#### 8. Try: Move `balance` column to be after `change` column

In [38]:
columns = list(savings_log.columns) # Create list of columns
columns.insert(columns.index("change")+1, columns.pop(columns.index("balance"))) # Move balance after change
savings_log = savings_log[columns] # Return dataframe with new column order

In [39]:
savings_log

,user_id,date,change,balance,notes
0,1,2024-01-01,15,15,Pellentesque viverra pede ac diam.
1,1,2024-01-01,39,34,"Lorem ipsum dolor sit amet, consectetuer adipi..."
2,1,2024-01-03,20,2,NaN
3,1,2024-01-10,95,3,Aenean sit amet justo.
4,1,2024-03-08,86,15,Pellentesque eget nunc.
...,...,...,...,...,...
1753,2,2025-08-23,-59,1966,"Maecenas leo odio, condimentum id, luctus nec,..."
1754,2,2025-08-23,21,1987,NaN
1755,2,2025-08-23,55,2042,NaN
1756,2,2025-08-23,-15,2027,NaN


#### 9. Sort resulting dataframe by date

In [40]:
# Sort by date
savings_log = savings_log.sort_values("date")

In [41]:
# Reset indexes
savings_log = savings_log.reset_index(drop=True)

In [42]:
savings_log

,user_id,date,change,balance,notes
0,1,2024-01-01,15,15,Pellentesque viverra pede ac diam.
1,2,2024-01-01,29,3,In sagittis dui vel nisl.
2,2,2024-01-01,25,25,Morbi porttitor lorem id ligula.
3,1,2024-01-01,39,34,"Lorem ipsum dolor sit amet, consectetuer adipi..."
4,1,2024-01-03,20,2,NaN
...,...,...,...,...,...
1753,1,2025-08-23,91,1526,Sed accumsan felis.
1754,2,2025-08-23,-87,2025,Maecenas rhoncus aliquam lacus.
1755,1,2025-08-24,8,1534,"Maecenas leo odio, condimentum id, luctus nec,..."
1756,1,2025-08-24,82,1616,Morbi vel lectus in quam fringilla rhoncus.


#### 10. Remove ";" from values in `notes` column

SchemaLoader in the main app splits sql statements by ';' so keeping it in notes values causes an error.

In [43]:
savings_log.loc[savings_log["notes"].notna(), "notes"] = (
    savings_log.loc[savings_log["notes"].notna(), "notes"]
    .str.replace(";", "", regex=False)
)

#### 11. Export dataframe to sql file with prefix `final_`

In [44]:
# Convert date column back to string
savings_log["date"] = savings_log["date"].dt.strftime("%Y-%m-%d")

In [45]:
# Write to sql file
with open("final_savings_log.sql", "w", encoding="UTF-8") as file:
    for _, row in savings_log.iterrows():
        notes = 'NULL' if str(row["notes"]) == 'nan' else f"'{row["notes"]}'"
        sql = f"INSERT INTO savings_log (user_id, date, change, balance, notes) VALUES({row["user_id"]}, '{row["date"]}', {row["change"]}, {row["balance"]}, {notes});\n"
        file.write(sql)

### users table

Write sql file containing two users each with `name`, `username`, and `password`

In [46]:
# Create users dictionary
users_dictionary = {
    "name": ["Yazeed", "Bara"],
    "username": ["admin", "bara"],
    "password": ["123456", "bara123"]
}

In [47]:
# Convert dictionary to dataframe
users = pd.DataFrame(users_dictionary)

In [48]:
users

,name,username,password
0,Yazeed,admin,123456
1,Bara,bara,bara123


In [49]:
# Write to sql file
with open("final_users.sql", "w", encoding="UTF-8") as file:
    for _, row in users.iterrows():
        sql = f"INSERT INTO users (name, username, password) VALUES('{row["name"]}', '{row["username"]}', '{row["password"]}');\n"
        file.write(sql)